In [1]:
import torch
import torch.nn as nn
import torchtext
from torchtext import data
from torchtext.vocab import Vectors
from torchtext.data import Iterator, BucketIterator
import spacy
import numpy as np


In [2]:
import jsonlines

In [6]:
with open('data/snli_1.0/snli_1.0_dev.jsonl', 'r+', encoding='utf-8') as f:
    for idx, item in enumerate(jsonlines.Reader(f)):
        print(item.keys())
        print(item.values())
        print(item)
        print((type(item)))
        if idx > 0:
            break

dict_keys(['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse'])
dict_values([['neutral', 'entailment', 'neutral', 'neutral', 'neutral'], '4705552913.jpg#2', 'neutral', '4705552913.jpg#2r1n', 'Two women are embracing while holding to go packages.', '( ( Two women ) ( ( are ( embracing ( while ( holding ( to ( go packages ) ) ) ) ) ) . ) )', '(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP are) (VP (VBG embracing) (SBAR (IN while) (S (NP (VBG holding)) (VP (TO to) (VP (VB go) (NP (NNS packages)))))))) (. .)))', 'The sisters are hugging goodbye while holding to go packages after just eating lunch.', '( ( The sisters ) ( ( are ( ( hugging goodbye ) ( while ( holding ( to ( ( go packages ) ( after ( just ( eating lunch ) ) ) ) ) ) ) ) ) . ) )', '(ROOT (S (NP (DT The) (NNS sisters)) (VP (VBP are) (VP (VBG hugging) (NP (UH goodbye)) (PP (IN while) (S (VP (VBG holding) (S (VP (T

In [2]:
spcay_load='en_core_web_sm'
nlp = spacy.load(spcay_load)

In [3]:
tokenizer_result = nlp.tokenizer('nlp is interesting')

In [4]:
type(tokenizer_result)

spacy.tokens.doc.Doc

In [5]:
doc_result = nlp('nlp is interesting')

In [6]:
type(doc_result)

spacy.tokens.doc.Doc

In [7]:
tokenizer_result == doc_result

False

In [8]:
tokenize = lambda x: [tok.text for tok in nlp.tokenizer(x) if not tok.is_punct | tok.is_space]

In [9]:
tokenize('nlp is interesting, isn\'t it?')

['nlp', 'is', 'interesting', 'is', "n't", 'it']

In [10]:
tokenize('nlp is interesting, hahaha!')

['nlp', 'is', 'interesting', 'hahaha']

In [21]:
vectors = Vectors(name='glove.840B.300d.txt')
vectors.unk_init = nn.init.xavier_uniform_

In [15]:
dir(vectors)[-10:]

['__str__',
 '__subclasshook__',
 '__weakref__',
 'cache',
 'dim',
 'get_vecs_by_tokens',
 'itos',
 'stoi',
 'unk_init',
 'vectors']

In [16]:
type(vectors.stoi)

dict

In [17]:
type(vectors.stoi)

dict

In [22]:
for idx, (key, value) in enumerate(vectors.stoi.items()):
    print('{key}  {value}'.format(key=key, value=value))
    if idx > 5:
        break

,  0
.  1
the  2
and  3
to  4
of  5
a  6


In [26]:
a = torch.rand((2, 3), requires_grad=True)

In [27]:
a.data

tensor([[0.8525, 0.7719, 0.1609],
        [0.3858, 0.2812, 0.3295]])

In [28]:
a

tensor([[0.8525, 0.7719, 0.1609],
        [0.3858, 0.2812, 0.3295]], requires_grad=True)

In [29]:
a.detach()

tensor([[0.8525, 0.7719, 0.1609],
        [0.3858, 0.2812, 0.3295]])

In [30]:
a.detach() == a.data

tensor([[True, True, True],
        [True, True, True]])

In [36]:
a.unsqueeze(1).shape

torch.Size([2, 1, 3])

In [31]:
drop_out_net = nn.Dropout()

In [34]:
drop_out_net.training

True

In [35]:
dir(drop_out_net)

['__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'half',
 'inplace',
 'load_